<a href="https://colab.research.google.com/github/shantanu2383/FAMA-FRENCH-5-FACTORS-ST-REVERSAL-LT-REVERSAL-MOMENTUM/blob/main/SS_CONSTRUCTING_FF3_%2B_FF5_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook takes the cleaned daily price data and fundamentals data, and constructs the relevant portfolios, before calculating the FAMA FRENCH 3 FACTORS.
The 3 factors are constructed first, then we construct the Investment and Operating Profitiability portfolios to construct the 5 factors

# CONSTRUCT FF3 FROM CLEANED PRICE AND FUNDAMENTALS DATA AND COMPARE TO FAMA FRENCH DAILY 3 FACTORS

In [ ]:
import pandas as pd
import statsmodels.formula.api as smf 
import numpy as np 
import matplotlib.pyplot as plt

!pip install pandasql
import os
from datetime import datetime
from sklearn.model_selection import train_test_split
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA 
from sklearn.metrics import mean_squared_error, mean_absolute_error 

import math
import matplotlib.pyplot as plt 
from datetime import datetime
import seaborn as sns 
import pandasql as ps

from sqlite3 import connect
conn=connect(':memory:')

from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26787 sha256=9578788fc3340e810e1b5f707d800ec483a8ff219a452a855bdb8b804e6b5284
  Stored in directory: /root/.cache/pip/wheels/ed/8f/46/a383923333728744f01ba24adbd8e364f2cb9470a8b8e5b9ff
Successfully built pandasql
Mounted at /content/gdrive


# IMPORT CLEANED DATAFRAME FROM DATA CLEANING NOTEBOOK WITH VALUE AND SIZE PORTFOLIOS

In [ ]:
file="cleaned_value_size_sorts.csv"
filepath="/content/gdrive/MyDrive/FIMA_2023_JANUARY/V2/DATALOAD/RAW_DATA/"
main_stocks_df=pd.read_csv(filepath + file)

FileNotFoundError: ignored

In [ ]:
main_stocks_df



```
# ADD IN MARKET CAP WEIGHT TO CALCULATE WEIGHTED RETURNS
```



In [ ]:
main_stocks_df['date']=pd.to_datetime(main_stocks_df['date'], format="%Y-%m-%d")
stock_weights=main_stocks_df[main_stocks_df['date'].dt.month==6]

stock_weights.to_sql('stock_weights', conn, if_exists='replace')

query='''
        SELECT DISTINCT ticker, quarter, mkt_cap, reference_date
        FROM stock_weights
        '''

stock_weights=pd.read_sql(query, conn)

#merge these weights back into main stocks df
main_stocks_df['reference_date']=pd.to_datetime(main_stocks_df['reference_date'], format="%Y-%m-%d")

stock_weights['reference_date']=pd.to_datetime(stock_weights['reference_date'], format="%Y-%m-%d")

main_stocks_df.to_sql('main_stocks_df', conn, if_exists='replace')
stock_weights.to_sql('stock_weights', conn, if_exists='replace')


query='''
        SELECT DISTINCT main_stocks_df.*, stock_weights.mkt_cap as mkt_cap_weight
        FROM main_stocks_df
        LEFT JOIN stock_weights
        ON main_stocks_df.ticker=stock_weights.ticker AND main_stocks_df.reference_date=stock_weights.reference_date
        '''

main_stocks_df_weights=pd.read_sql(query, conn)

main_stocks_df=main_stocks_df_weights

/usr/local/lib/python3.8/dist-packages/pandas/core/generic.py:2872: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


# CALCULATE WEIGHTED RETURNS

In [ ]:
main_stocks_df['date']=pd.to_datetime(main_stocks_df['date'])

valueWeightRet=main_stocks_df.groupby(['date', 'size_portfolio', 'value_portfolio']).apply(lambda x: np.average(pd.to_numeric(x['ret']), weights=pd.to_numeric(x['mkt_cap_weight'])))

'''
The groupby method groups the rows of the stocks_18_22 DataFrame by the values in the date, size_portfolio, and value_portfolio columns. 
This creates a GroupBy object, which is an intermediate object that represents the groups.

The apply method is called on the GroupBy object with a lambda function as its argument. 
This lambda function takes a DataFrame as input (which represents one of the groups) and returns a single value.

The lambda function calculates the weighted average of the ret column in the input DataFrame using the market_cap_weight column as the weights. 
It does this by calling the np.average function and passing the ret column and the market_cap_weight column as the weights argument. 
It also converts the ret and market_cap_weight columns to numeric using the pd.to_numeric function.

The apply method applies the lambda function to each group and returns a new Series with the calculated weighted averages. 
The index of the Series will have the same levels as the original GroupBy object, which are the date, size_portfolio,
 and value_portfolio columns, and the values will be the weighted averages of the ret column for each group.
 '''

In [ ]:
#Create factors dataframe
main_stocks_df.to_sql("main_stocks_df", conn, if_exists='replace')

query=''' 
        SELECT DISTINCT date
        FROM main_stocks_df
        '''
factors=pd.read_sql(query, conn)
factors['date']=pd.to_datetime(factors['date'])



In [ ]:
#Add in the three factors (SMB, HML, MKT)

#Add SMB portfolios:
factors['smb']=""
for i in factors.index:
  dateofData=factors['date'][i]
  factors['smb'][i]=np.average(valueWeightRet[dateofData]["S"])-np.average(valueWeightRet[dateofData]["B"])
                                                                           

'''
 It assigns the value of the date column for the current row to the dateofData variable.
It assigns the result of the following calculation to the smb column for the current row:
np.average(valueWeightRet[dateofData]["S"]): Calculates the average of the values in the valueWeightRet Series for the index level with the value dateofData and the size_portfolio level with the value "S".
np.average(valueWeightRet[dateofData]["B"]): Calculates the average of the values in the valueWeightRet Series for the index level with the value dateofData and the size_portfolio level with the value "B".
The difference between these two averages is then assigned to the smb column.       
'''             

In [ ]:
#Change index of valueWeightedRet for HML

valueWeightRet=main_stocks_df.groupby(['date', 'value_portfolio', 'size_portfolio']).apply(lambda x: np.average(pd.to_numeric(x['ret']), weights=pd.to_numeric(x['mkt_cap_weight'])))

#Add HML portfolios
factors['hml']=""

for i in factors.index:
  dateofData=factors['date'][i]
  factors['hml'][i]=np.average(valueWeightRet[dateofData]["H"])-np.average(valueWeightRet[dateofData]["L"])



<ipython-input-8-2918a717a1f7>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  factors['hml'][i]=np.average(valueWeightRet[dateofData]["H"])-np.average(valueWeightRet[dateofData]["L"])


In [ ]:
#Add mkt (monthly average returns) across all stocks

factors['mkt']=""
main_stocks_df.dropna(inplace=True)
weightedAvgMonthlyRet=main_stocks_df.groupby('date').apply(lambda x: np.average(pd.to_numeric(x.ret), weights=pd.to_numeric(x.mkt_cap_weight)))

for i in factors.index:
  dateofData=factors['date'][i]
  factors['mkt'][i]=weightedAvgMonthlyRet[dateofData]

In [ ]:
main_stocks_df=main_stocks_df.fillna(0)
#Add mkt ( daily returns) across all stocks
factors['mkt']=""
main_stocks_df.dropna(inplace=True)
weighted_dailyRet=main_stocks_df.groupby('date').apply(lambda x: np.average(pd.to_numeric(x.ret), weights=pd.to_numeric(x.mkt_cap_weight)))

for i in factors.index:
  dateofData=factors['date'][i]
  factors['mkt'][i]=weighted_dailyRet[dateofData]

<ipython-input-9-8fb2179436e5>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  factors['mkt'][i]=weighted_dailyRet[dateofData]


In [ ]:
factors

,date,smb,hml,mkt
0,2019-06-03,0.428143,2.394259,-0.859617
1,2019-06-04,0.692062,0.142885,2.296644
2,2019-06-05,-0.704983,-1.232508,0.691613
3,2019-06-06,-1.116536,0.155471,0.620872
4,2019-06-07,0.128809,-0.916262,1.465795
...,...,...,...,...
772,2022-06-24,-1.24036,0.111596,3.215851
773,2022-06-27,0.08939,1.115815,-0.392768
774,2022-06-28,-0.60101,1.371554,-2.361435
775,2022-06-29,-0.903176,-0.944274,0.0175


\# READ IN KENNETH FRENCH DATA FOR COMPARISON

In [ ]:
import pandas_datareader.data as web
from pandas_datareader.famafrench import get_available_datasets
datasets = get_available_datasets()
df_3_factor=[dataset for dataset in datasets if 'Research' in dataset and 'Factor' in dataset]

In [ ]:
df_3_factor
ff=web.DataReader(df_3_factor[2],'famafrench',start='2018-07-01',end='2022-11-01')[0]
ff['dd']=ff.index
ff['dd']=pd.to_datetime(ff['dd'], format="%Y-%m-%d")
factors['date']=pd.to_datetime(factors['date'])
ff=ff[ff['dd']>="2019-06-03"]

In [ ]:
ff

,Mkt-RF,SMB,HML,RF,dd
Date,,,,,
2019-06-03,-0.40,0.35,1.65,0.009,2019-06-03
2019-06-04,2.33,0.42,-0.02,0.009,2019-06-04
2019-06-05,0.70,-0.98,-0.97,0.009,2019-06-05
2019-06-06,0.55,-1.04,0.01,0.009,2019-06-06
2019-06-07,1.04,0.05,-1.20,0.009,2019-06-07
...,...,...,...,...,...
2022-10-26,-0.66,0.42,0.89,0.011,2022-10-26
2022-10-27,-0.54,0.32,1.06,0.011,2022-10-27
2022-10-28,2.33,-0.04,-0.34,0.011,2022-10-28


In [ ]:
ff=ff.rename(columns={'Mkt-RF': 'MKT'})

In [ ]:
ff.to_sql('ff', conn, if_exists='replace')

query='''
        SELECT DISTINCT dd as date, SMB as SMB_ff, HML as HML_ff, MKT as MKT_ff
        FROM FF
        '''
ff=pd.read_sql(query, conn)

In [ ]:
ff['date']=pd.to_datetime(ff['date'], errors='coerce')

x=["SMB_ff", "HML_ff"]
for x in x:
  ff[x]=pd.to_numeric(ff[x], errors='coerce')

y=['smb', 'hml']

for y in y:
  factors[y]=pd.to_numeric(factors[y], errors='coerce')

In [ ]:
ff

,date,SMB_ff,HML_ff,MKT_ff
0,2019-06-03 00:00:00,0.35,1.65,-0.40
1,2019-06-04 00:00:00,0.42,-0.02,2.33
2,2019-06-05 00:00:00,-0.98,-0.97,0.70
3,2019-06-06 00:00:00,-1.04,0.01,0.55
4,2019-06-07 00:00:00,0.05,-1.20,1.04
...,...,...,...,...
858,2022-10-26 00:00:00,0.42,0.89,-0.66
859,2022-10-27 00:00:00,0.32,1.06,-0.54
860,2022-10-28 00:00:00,-0.04,-0.34,2.33
861,2022-10-31 00:00:00,0.39,0.75,-0.67


In [ ]:
factors.to_sql('factors', conn, if_exists='replace')
ff.to_sql('ff', conn, if_exists='replace')

query='''
        SELECT factors.date, factors.smb, factors.hml, factors.mkt, FF.SMB_ff, FF.HML_ff, FF.MKT_ff
        FROM factors
        LEFT JOIN ff
        ON factors.date=ff.date
        '''

ff_checker=pd.read_sql(query, conn)

# COMPARE RESULTS 

In [ ]:
ff_checker

,date,smb,hml,mkt,SMB_ff,HML_ff,MKT_ff
0,2019-06-03 00:00:00,0.428143,2.394259,-0.859617,0.35,1.65,-0.40
1,2019-06-04 00:00:00,0.692062,0.142885,2.296644,0.42,-0.02,2.33
2,2019-06-05 00:00:00,-0.704983,-1.232508,0.691613,-0.98,-0.97,0.70
3,2019-06-06 00:00:00,-1.116536,0.155471,0.620872,-1.04,0.01,0.55
4,2019-06-07 00:00:00,0.128809,-0.916262,1.465795,0.05,-1.20,1.04
...,...,...,...,...,...,...,...
772,2022-06-24 00:00:00,-1.240360,0.111596,3.215851,-0.37,-0.04,3.11
773,2022-06-27 00:00:00,0.089390,1.115815,-0.392768,0.57,1.24,-0.28
774,2022-06-28 00:00:00,-0.601010,1.371554,-2.361435,-0.35,2.36,-2.10
775,2022-06-29 00:00:00,-0.903176,-0.944274,0.017500,-0.45,-1.30,-0.20


In [ ]:
ff_checker.describe(percentiles=[0.1, 0.25, 0.5, 0.75, 0.9])

,smb,hml,mkt,SMB_ff,HML_ff,MKT_ff
count,777.000000,777.000000,777.000000,777.000000,777.000000,777.000000
mean,0.009582,0.034096,0.071871,0.000875,0.012008,0.054003
std,0.910469,1.237980,1.588995,0.785899,1.296899,1.538097
min,-3.764038,-6.040538,-11.823214,-3.570000,-5.000000,-12.000000
10%,-1.054447,-1.337060,-1.620918,-0.884000,-1.490000,-1.530000
25%,-0.551783,-0.668820,-0.521912,-0.470000,-0.710000,-0.520000
50%,-0.041632,-0.020256,0.119769,-0.020000,-0.040000,0.120000
75%,0.526672,0.670252,0.855271,0.470000,0.700000,0.800000
90%,1.121418,1.600312,1.532367,0.930000,1.602000,1.458000
max,7.894626,6.935453,9.909721,5.500000,6.740000,9.340000


In [ ]:
ff_checker.corr(method='pearson')

,smb,hml,mkt,SMB_ff,HML_ff,MKT_ff
smb,1.000000,0.071423,0.276065,0.912444,0.115027,0.285777
hml,0.071423,1.000000,-0.108276,0.096985,0.906232,-0.029970
mkt,0.276065,-0.108276,1.000000,0.168191,-0.061345,0.990936
SMB_ff,0.912444,0.096985,0.168191,1.000000,0.064708,0.177203
HML_ff,0.115027,0.906232,-0.061345,0.064708,1.000000,0.021526
MKT_ff,0.285777,-0.029970,0.990936,0.177203,0.021526,1.000000


# CONSTRUCT FAMA FRENCH 5 FACTORS

In [ ]:
'''
Here we update our code to caluclate the FF5 factors
Since we already have a clean dataframe with value and size portfolios, and cleaned fundamental data 
our methodology is as follows:
1. IMPORT CLEANED STOCKS DATAFRAME AND CLEANED FUNDAMENTALS DATA
2. EXTRACT END OF YEAR INVESTMENT AND OPERATING PROFITABILTY DATA FROM CLEANED FUNDAMENTALS DATA
3. MERGE BACK INTO MAIN STOCKS DATAFRAME
4. CALCULATE BREAKPOINTS AND SORT INTO INVESTMENT AND PROFITABILITY PORTFOLIOS
5. CONSTRUCT RELEVANT PORTFOLIOS
6. CALCULATE FF5 FACTORS
'''

# IMPORT CLEANED DATA

In [ ]:
import pandas as pd
import statsmodels.formula.api as smf 
import numpy as np 
import matplotlib.pyplot as plt

!pip install pandasql
import os
from datetime import datetime
from sklearn.model_selection import train_test_split
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA 
from sklearn.metrics import mean_squared_error, mean_absolute_error 

import math
import matplotlib.pyplot as plt 
from datetime import datetime
import seaborn as sns 
import pandasql as ps

from sqlite3 import connect
conn=connect(':memory:')

from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26787 sha256=b79f5ac50344c0b0689005e45bbdb2d01d2ea6e661d3c786a0ee7137995e1d47
  Stored in directory: /root/.cache/pip/wheels/ed/8f/46/a383923333728744f01ba24adbd8e364f2cb9470a8b8e5b9ff
Successfully built pandasql
Mounted at /content/gdrive


In [ ]:
#import stocks dataframe used to calculate FF3
file="cleaned_value_size_sorts.csv"
filepath="/content/gdrive/MyDrive/Fundamentals/"
main_stocks_df=pd.read_csv(filepath + file)

In [ ]:
main_stocks_df

,Unnamed: 0,ticker,date,ret,quarter,mkt_cap,reference_date,book_value,mkt_equity,bm_ratio,exchange,size_median,size_portfolio,value_portfolio,value_q30,value_q70
0,0,AA,2019-06-03 00:00:00,1.510146,2,3724.45,2019-06-01 00:00:00,7588.0000,5224.02,1.452521,NYSE,2958.73,B,H,0.211176,0.603634
1,1,AA,2019-06-04 00:00:00,1.766620,2,3724.45,2019-06-01 00:00:00,7588.0000,5224.02,1.452521,NYSE,2958.73,B,H,0.211176,0.603634
2,2,AA,2019-06-05 00:00:00,-2.375514,2,3724.45,2019-06-01 00:00:00,7588.0000,5224.02,1.452521,NYSE,2958.73,B,H,0.211176,0.603634
3,3,AA,2019-06-06 00:00:00,0.935891,2,3724.45,2019-06-01 00:00:00,7588.0000,5224.02,1.452521,NYSE,2958.73,B,H,0.211176,0.603634
4,4,AA,2019-06-07 00:00:00,-0.602689,2,3724.45,2019-06-01 00:00:00,7588.0000,5224.02,1.452521,NYSE,2958.73,B,H,0.211176,0.603634
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1815571,1815571,ZYNE,2022-06-24 00:00:00,4.166667,2,33.49,2022-06-01 00:00:00,70.9133,89.42,0.793036,NASDAQ,2250.02,S,H,0.145164,0.486365
1815572,1815572,ZYNE,2022-06-27 00:00:00,-0.800000,2,33.49,2022-06-01 00:00:00,70.9133,89.42,0.793036,NASDAQ,2250.02,S,H,0.145164,0.486365
1815573,1815573,ZYNE,2022-06-28 00:00:00,-0.806452,2,33.49,2022-06-01 00:00:00,70.9133,89.42,0.793036,NASDAQ,2250.02,S,H,0.145164,0.486365
1815574,1815574,ZYNE,2022-06-29 00:00:00,-6.504065,2,33.49,2022-06-01 00:00:00,70.9133,89.42,0.793036,NASDAQ,2250.02,S,H,0.145164,0.486365


In [ ]:
#Import cleaned fundamentals data 
filepath="/content/gdrive/MyDrive/clean/"
fundamentals=pd.read_csv(filepath + "cleaned_fundamentals_updated.csv")

In [ ]:
fundamentals

,Unnamed: 0,index,ticker,exchange,tot_asset,net_income_loss_share_holder,comm_shares_out,per_end_date,per_type,tot_liab,book_val_per_share,mkt_val,operating_profitability,book_value
0,26,37,AAC,NYSE,1002.156,20.3805,31.9080,2021-03-31 00:00:00,Q,997.1564,0.1567,315.89,4.076426,4.9996
1,27,38,A,NYSE,7872.000,168.0000,322.0000,2017-01-31 00:00:00,Q,3572.0000,13.3540,NaN,0.039070,4300.0000
2,28,39,A,NYSE,8016.000,164.0000,321.0000,2017-04-30 00:00:00,Q,3641.0000,13.6293,NaN,0.037486,4375.0000
3,29,40,A,NYSE,8261.000,175.0000,322.0000,2017-07-31 00:00:00,Q,3650.0000,14.3199,NaN,0.037953,4611.0000
4,30,41,A,NYSE,8426.000,177.0000,322.0000,2017-10-31 00:00:00,Q,3591.0000,15.0155,NaN,0.036608,4835.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107192,166872,208055,ZYXI,NASDAQ,90.723,6.1070,38.2514,2021-09-30 00:00:00,Q,27.3400,1.6570,NaN,0.096351,63.3830
107193,166873,208056,ZYXI,NASDAQ,133.274,8.8940,39.7380,2021-12-31 00:00:00,Q,59.3490,1.8603,NaN,0.120311,73.9250
107194,166874,208057,ZYXI,NASDAQ,132.041,1.3770,39.7770,2022-03-31 00:00:00,Q,56.2230,1.9061,NaN,0.018162,75.8180
107195,166875,208058,ZYXI,NASDAQ,119.575,3.3460,38.4040,2022-06-30 00:00:00,Q,50.5660,1.7969,NaN,0.048486,69.0090


In [ ]:
#extract variables for investment and operating_profitability

fundamentals.to_sql('f', conn, if_exists='replace')

query='''
        SELECT DISTINCT ticker, exchange, per_end_date as date, operating_profitability as op, tot_asset as at
        FROM f
        '''

inv_op=pd.read_sql(query, conn)

/usr/local/lib/python3.8/dist-packages/pandas/core/generic.py:2872: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [ ]:
inv_op

,ticker,exchange,date,op,at
0,AAC,NYSE,2021-03-31 00:00:00,4.076426,1002.156
1,A,NYSE,2017-01-31 00:00:00,0.039070,7872.000
2,A,NYSE,2017-04-30 00:00:00,0.037486,8016.000
3,A,NYSE,2017-07-31 00:00:00,0.037953,8261.000
4,A,NYSE,2017-10-31 00:00:00,0.036608,8426.000
...,...,...,...,...,...
107192,ZYXI,NASDAQ,2021-09-30 00:00:00,0.096351,90.723
107193,ZYXI,NASDAQ,2021-12-31 00:00:00,0.120311,133.274
107194,ZYXI,NASDAQ,2022-03-31 00:00:00,0.018162,132.041
107195,ZYXI,NASDAQ,2022-06-30 00:00:00,0.048486,119.575


In [ ]:
#keep last  observation of each year i.e Q4

inv_op['date']=pd.to_datetime(inv_op['date'], format='%Y-%m-%d')
inv_op['quarter']=inv_op['date'].dt.quarter
inv_op['month']=inv_op['date'].dt.month
inv_op=inv_op[inv_op['month']==12]
inv_op

<ipython-input-10-3cd24665ba25>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inv_op['date']=pd.to_datetime(inv_op['date'], format='%Y-%m-%d')
<ipython-input-10-3cd24665ba25>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inv_op['quarter']=inv_op['date'].dt.quarter
<ipython-input-10-3cd24665ba25>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

,ticker,exchange,date,op,at,quarter,month
28,AA,NYSE,2017-12-31,-0.019712,17447.000,4,12
33,AA,NYSE,2018-12-31,0.006721,16132.000,4,12
37,AA,NYSE,2019-12-31,-0.051478,14631.000,4,12
41,AA,NYSE,2020-12-31,-0.000797,14860.000,4,12
45,AA,NYSE,2021-12-31,-0.062381,15025.000,4,12
...,...,...,...,...,...,...,...
107177,ZYXI,NASDAQ,2017-12-31,0.674827,8.929,4,12
107181,ZYXI,NASDAQ,2018-12-31,0.282316,19.251,4,12
107185,ZYXI,NASDAQ,2019-12-31,0.149952,28.277,4,12
107189,ZYXI,NASDAQ,2020-12-31,0.031431,72.175,4,12


In [ ]:
#calculate investment

inv_op['at_t_minus_1']=inv_op['at'].shift(1) #get investment in previous year

inv_op['stock_checker']=inv_op['ticker'].shift(1)

inv_op=inv_op[(inv_op['ticker'])==inv_op['stock_checker']] #deletes first stock of each yearly series

inv_op['inv']=inv_op['at']/inv_op['at_t_minus_1']
               



<ipython-input-11-91f936cbc7d0>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inv_op['at_t_minus_1']=inv_op['at'].shift(1) #get investment in previous year
<ipython-input-11-91f936cbc7d0>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inv_op['stock_checker']=inv_op['ticker'].shift(1)
<ipython-input-11-91f936cbc7d0>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

In [ ]:
inv_op

,ticker,exchange,date,op,at,quarter,month,at_t_minus_1,stock_checker,inv
33,AA,NYSE,2018-12-31,0.006721,16132.0000,4,12,17447.0000,AA,0.924629
37,AA,NYSE,2019-12-31,-0.051478,14631.0000,4,12,16132.0000,AA,0.906955
41,AA,NYSE,2020-12-31,-0.000797,14860.0000,4,12,14631.0000,AA,1.015652
45,AA,NYSE,2021-12-31,-0.062381,15025.0000,4,12,14860.0000,AA,1.011104
54,AAC,NYSE,2021-12-31,0.062351,1001.5710,4,12,0.5235,AAC,1913.220630
...,...,...,...,...,...,...,...,...,...,...
107170,ZYNE,NASDAQ,2021-12-31,-0.124308,81.1715,4,12,74.0568,ZYNE,1.096071
107181,ZYXI,NASDAQ,2018-12-31,0.282316,19.2510,4,12,8.9290,ZYXI,2.156009
107185,ZYXI,NASDAQ,2019-12-31,0.149952,28.2770,4,12,19.2510,ZYXI,1.468859
107189,ZYXI,NASDAQ,2020-12-31,0.031431,72.1750,4,12,28.2770,ZYXI,2.552428


In [ ]:
inv_op['date']=pd.to_datetime(inv_op['date'])
inv_op['year']=inv_op['date'].dt.year

inv_op['year']=pd.to_numeric(inv_op['year'], errors='coerce')

#ADD REFERENCE DATE FOR MATCHING (6/1 of next calendar year)
inv_op['reference_date']=inv_op['year']+1
inv_op['reference_date']=inv_op['reference_date'].apply(str)
inv_op['reference_date'] = inv_op['reference_date'].apply(lambda x: x + "-06-01")
inv_op['reference_date']=pd.to_datetime(inv_op['reference_date'])
inv_op.drop('year', inplace=True, axis=1)

<ipython-input-13-445d31e363e9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inv_op['date']=pd.to_datetime(inv_op['date'])
<ipython-input-13-445d31e363e9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inv_op['year']=inv_op['date'].dt.year
<ipython-input-13-445d31e363e9>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable

In [ ]:
#join back into main stocks dataframe

inv_op.to_sql('a', conn, if_exists='replace')
main_stocks_df.to_sql('main', conn, if_exists='replace')


query=''' 
          SELECT DISTINCT main.*, a.op, a.inv
          FROM main
          LEFT JOIN a
          ON main.reference_date=a.reference_date AND main.ticker=A.ticker'''

main_stocks_df_inv_op=pd.read_sql(query, conn)

/usr/local/lib/python3.8/dist-packages/pandas/core/generic.py:2872: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [ ]:
main_stocks_df=main_stocks_df_inv_op

# NOW WE CAN DO OUR ROBUSTNESS AND INVESTMENT SORTS

In [ ]:
main_stocks_df['date']=pd.to_datetime(main_stocks_df['date'])
main_stocks_df.replace([np.inf, -np.inf], np.nan, inplace=True)
main_stocks_df.dropna(inplace=True)
robust_breakpoints=main_stocks_df[(main_stocks_df['date'].dt.month==6) & (main_stocks_df['exchange']=="NYSE")]

robust_breakpoints.to_sql('robust', conn, if_exists='replace')

query='''
        SELECT DISTINCT ticker, reference_date, OP
        FROM robust
        '''

robust_breakpoints=pd.read_sql(query, conn)
robust_breakpoints['reference_date']=pd.to_datetime(robust_breakpoints['reference_date'])

In [ ]:
g=robust_breakpoints.groupby(['reference_date'])

In [ ]:
robust_sorts=main_stocks_df[(main_stocks_df['date'].dt.month==6)]
robust_sorts.replace([np.inf, -np.inf], np.nan, inplace=True)
robust_sorts.dropna(inplace=True)

In [ ]:
robust_sorts

In [ ]:
robust_sorts['robust_q30']=""
robust_sorts['robust_q70']=""
c=0
for i in robust_sorts.index:
  robust_sorts['robust_q30'][i]=g.get_group(g.get_group(robust_sorts['reference_date'][i]).reset_index()['reference_date'][0])['op'].quantile(0.3)
  robust_sorts['robust_q70'][i]=g.get_group(g.get_group(robust_sorts['reference_date'][i]).reset_index()['reference_date'][0])['op'].quantile(0.7)
  c=c+1
  print(c)


In [ ]:
#export for sense-check
filepath="/content/gdrive/MyDrive/"
file='robust-sorts.csv'

robust_sorts.to_csv(filepath + file)

In [ ]:
robust_sorts["robust_portfolio"]=""
for i in robust_sorts.index:
  if robust_sorts['op'][i] > robust_sorts['robust_q70'][i]:
    robust_sorts['robust_portfolio'][i]="R"
  elif robust_sorts['robust_q30'][i] < robust_sorts['op'][i] <= robust_sorts['robust_q70'][i]:
    robust_sorts['robust_portfolio'][i]="N"
  else:
    robust_sorts['robust_portfolio'][i]="W"

# INVESTMENT SORT

In [ ]:
main_stocks_df['date']=pd.to_datetime(main_stocks_df['date'])

In [ ]:
main_stocks_df.replace([np.inf, -np.inf], np.nan, inplace=True)
main_stocks_df.dropna(inplace=True)
investment_breakpoints=main_stocks_df[(main_stocks_df['date'].dt.month==6) & (main_stocks_df['exchange']=="NYSE") & (main_stocks_df['inv'] != "")]


In [ ]:
investment_breakpoints.to_sql("investment_breakpoints", conn, if_exists='replace')

query='''
        SELECT DISTINCT reference_date, ticker, exchange, inv 
        FROM investment_breakpoints
        '''

investment_breakpoints=pd.read_sql(query, conn)

/usr/local/lib/python3.8/dist-packages/pandas/core/generic.py:2872: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [ ]:
investment_breakpoints['inv']=pd.to_numeric(investment_breakpoints['inv'], errors='coerce')
investment_breakpoints
investment_breakpoints['reference_date']=pd.to_datetime(investment_breakpoints['reference_date'])
g=investment_breakpoints.groupby(['reference_date'])

In [ ]:
investment_sorts=main_stocks_df[(main_stocks_df['date'].dt.month==6) & (main_stocks_df['inv'] != "")]

investment_sorts['inv']=pd.to_numeric(investment_sorts['inv'], errors='coerce')
main_stocks_df.replace([np.inf, -np.inf], np.nan, inplace=True)
main_stocks_df.dropna(inplace=True)

<ipython-input-35-785d739cb57f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  investment_sorts['inv']=pd.to_numeric(investment_sorts['inv'], errors='coerce')


In [ ]:
#calculate 30th and 70th quantile of investments
investment_sorts['investment_q30']=""
investment_sorts['investment_q70']=""

for i in investment_sorts.index:
  investment_sorts['investment_q30'][i]=g.get_group(g.get_group(investment_sorts['reference_date'][i]).reset_index()['reference_date'][0])['inv'].quantile(0.3)
  investment_sorts['investment_q70'][i]=g.get_group(g.get_group(investment_sorts['reference_date'][i]).reset_index()['reference_date'][0])['inv'].quantile(0.7)


<ipython-input-36-873f5c047035>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  investment_sorts['investment_q30']=""
<ipython-input-36-873f5c047035>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  investment_sorts['investment_q70']=""
<ipython-input-36-873f5c047035>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  invest

In [ ]:
#export for sense-check
filepath="/content/gdrive/MyDrive/"
file='inv-sorts.csv'

investment_sorts.to_csv(filepath + file)

In [ ]:
investment_sorts

,index,Unnamed: 0,ticker,date,ret,quarter,mkt_cap,reference_date,book_value,mkt_equity,...,exchange,size_median,size_portfolio,value_portfolio,value_q30,value_q70,op,inv,investment_q30,investment_q70
0,0,0,AA,2019-06-03,1.510146,2,3724.45,2019-06-01 00:00:00,7588.0000,5224.02,...,NYSE,2958.73,B,H,0.211176,0.603634,0.006721,0.924629,0.983478,1.083023
1,1,1,AA,2019-06-04,1.766620,2,3724.45,2019-06-01 00:00:00,7588.0000,5224.02,...,NYSE,2958.73,B,H,0.211176,0.603634,0.006721,0.924629,0.983478,1.083023
2,2,2,AA,2019-06-05,-2.375514,2,3724.45,2019-06-01 00:00:00,7588.0000,5224.02,...,NYSE,2958.73,B,H,0.211176,0.603634,0.006721,0.924629,0.983478,1.083023
3,3,3,AA,2019-06-06,0.935891,2,3724.45,2019-06-01 00:00:00,7588.0000,5224.02,...,NYSE,2958.73,B,H,0.211176,0.603634,0.006721,0.924629,0.983478,1.083023
4,4,4,AA,2019-06-07,-0.602689,2,3724.45,2019-06-01 00:00:00,7588.0000,5224.02,...,NYSE,2958.73,B,H,0.211176,0.603634,0.006721,0.924629,0.983478,1.083023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1815571,1815571,1815571,ZYNE,2022-06-24,4.166667,2,33.49,2022-06-01 00:00:00,70.9133,89.42,...,NASDAQ,2250.02,S,H,0.145164,0.486365,-0.124308,1.096071,1.012654,1.190353
1815572,1815572,1815572,ZYNE,2022-06-27,-0.800000,2,33.49,2022-06-01 00:00:00,70.9133,89.42,...,NASDAQ,2250.02,S,H,0.145164,0.486365,-0.124308,1.096071,1.012654,1.190353
1815573,1815573,1815573,ZYNE,2022-06-28,-0.806452,2,33.49,2022-06-01 00:00:00,70.9133,89.42,...,NASDAQ,2250.02,S,H,0.145164,0.486365,-0.124308,1.096071,1.012654,1.190353
1815574,1815574,1815574,ZYNE,2022-06-29,-6.504065,2,33.49,2022-06-01 00:00:00,70.9133,89.42,...,NASDAQ,2250.02,S,H,0.145164,0.486365,-0.124308,1.096071,1.012654,1.190353


In [ ]:
#sort investment into conservative, neutral, and aggressive portfolios
investment_sorts['investment_portfolio']=""
for i in investment_sorts.index:
  if investment_sorts['inv'][i]>investment_sorts['investment_q70'][i]:
    investment_sorts['investment_portfolio'][i]="A"
  elif investment_sorts['investment_q30'][i] < investment_sorts['inv'][i] <= investment_sorts['investment_q70'][i]:
    investment_sorts['investment_portfolio'][i]="N"
  else:
    investment_sorts['investment_portfolio'][i]="C"

<ipython-input-39-4bad7c55eda8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  investment_sorts['investment_portfolio']=""
<ipython-input-39-4bad7c55eda8>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  investment_sorts['investment_portfolio'][i]="C"
<ipython-input-39-4bad7c55eda8>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  investment_sorts['investment_portfolio'][i]="N"

In [ ]:
robust_sorts

NameError: ignored

In [ ]:
#import stocks dataframe used to calculate FF3
file="cleaned_value_size_sorts.csv"
filepath="/content/gdrive/MyDrive/Fundamentals/"
main_stocks_df=pd.read_csv(filepath + file)

#merge investment and robust sorts back into main df

x=[main_stocks_df, investment_sorts, robust_sorts]
for x in x:
  x['date']=pd.to_datetime(x['date'])

main_stocks_df.to_sql('main', conn, if_exists='replace')
investment_sorts.to_sql('inv', conn, if_exists='replace')
robust_sorts.to_sql('robust', conn, if_exists='replace')

query='''SELECT DISTINCT main.*, inv.investment_portfolio, robust.robust_portfolio
          FROM main
          LEFT JOIN inv on main.reference_date=inv.reference_date AND main.ticker=inv.ticker
          LEFT JOIN robust on main.reference_date=robust.reference_date AND main.ticker=robust.ticker
          '''

main_stocks_df_x=pd.read_sql(query, conn)

In [ ]:
main_stocks_df=main_stocks_df_x


In [ ]:
main_stocks_df

In [ ]:
#get june market cap for weights

main_stocks_df['date']=pd.to_datetime(main_stocks_df['date'], format="%Y-%m-%d")
stock_weights=main_stocks_df[main_stocks_df['date'].dt.month==6]

#stock_weights=stock_weights.drop(['level_0', 'index'], axis=1)
#main_stocks_df=main_stocks_df.drop(['level_0', 'index'], axis=1)

stock_weights.to_sql('stock_weights', conn, if_exists='replace')

query='''
        SELECT DISTINCT ticker, quarter, mkt_cap, reference_date
        FROM stock_weights
        '''

stock_weights=pd.read_sql(query, conn)

#merge these weights back into main stocks df
main_stocks_df['reference_date']=pd.to_datetime(main_stocks_df['reference_date'], format="%Y-%m-%d")

stock_weights['reference_date']=pd.to_datetime(stock_weights['reference_date'], format="%Y-%m-%d")

main_stocks_df.to_sql('main_stocks_df', conn, if_exists='replace')
stock_weights.to_sql('stock_weights', conn, if_exists='replace')


query='''
        SELECT DISTINCT main_stocks_df.*, stock_weights.mkt_cap as mkt_cap_weight
        FROM main_stocks_df
        LEFT JOIN stock_weights
        ON main_stocks_df.ticker=stock_weights.ticker AND main_stocks_df.reference_date=stock_weights.reference_date
        '''

main_stocks_df_weights=pd.read_sql(query, conn)

main_stocks_df=main_stocks_df_weights

In [ ]:
main_stocks_df

# CONSTRUCT PORTFOLIOS


Construction:	 	The Fama/French 5 factors (2x3) are constructed using the 6 value-weight portfolios formed on size and book-to-market, the 6 value-weight portfolios formed on size and operating profitability, and the 6 value-weight portfolios formed on size and investment. (See the description of the 6 size/book-to-market, size/operating profitability, size/investment portfolios.)
 	 	 

# SIZE OP PORTFOLIO

In [ ]:
main_stocks_df.to_csv(filepath + "cleaned stocks with all portfolios.csv")

In [ ]:
##CONSTRUCT SIZE/OP PORTFOLIO
main_stocks_df['date']=pd.to_datetime(main_stocks_df['date'])
valueWeightRet_SIZE_OP=main_stocks_df.groupby(['date', 'robust_portfolio', 'size_portfolio']).apply(lambda x: np.average(pd.to_numeric(x['ret']), weights=pd.to_numeric(x['mkt_cap'])))


In [ ]:
valueWeightRet_SIZE_OP=pd.DataFrame(valueWeightRet_SIZE_OP)

In [ ]:
valueWeightRet_SIZE_OP

In [ ]:
valueWeightRet_SIZE_OP=valueWeightRet_SIZE_OP.reset_index(level=['size_portfolio', 'robust_portfolio'])

In [ ]:
x=['size_portfolio', 'robust_portfolio']

for x in x:
  valueWeightRet_SIZE_OP[x]=valueWeightRet_SIZE_OP[x].apply(str)

  

In [ ]:
portfolio_SIZE_OP=valueWeightRet_SIZE_OP

In [ ]:
portfolio_SIZE_OP['size_robust_portfolio']=portfolio_SIZE_OP['size_portfolio'] + "/" + portfolio_SIZE_OP['robust_portfolio'] 


In [ ]:
portfolio_SIZE_OP=portfolio_SIZE_OP.drop(['size_portfolio', 'robust_portfolio'], axis=1)

In [ ]:
portfolio_SIZE_OP=pd.pivot(portfolio_SIZE_OP, columns='size_robust_portfolio', values=0)

In [ ]:
portfolio_SIZE_OP.columns=['BIG_NEUTRAL', 'BIG_ROBUST', 'BIG_WEAK', 'SMALL_NEUTRAL', 'SMALL_ROBUST', 'SMALL_WEAK']

In [ ]:
#Calculate RMW FACTOR

portfolio_SIZE_OP['RMW']=0.5 * (portfolio_SIZE_OP['SMALL_ROBUST'] + portfolio_SIZE_OP['BIG_ROBUST']) - 0.5 * (portfolio_SIZE_OP['SMALL_WEAK'] + portfolio_SIZE_OP['BIG_WEAK'])

In [ ]:
portfolio_SIZE_OP

# SIZE-INVESTMENT PORTFOLIOS

In [ ]:
##CONSTRUCT SIZE/OP PORTFOLIO

valueWeightRet_SIZE_INV=main_stocks_df.groupby(['date', 'investment_portfolio', 'size_portfolio']).apply(lambda x: np.average(pd.to_numeric(x['ret']), weights=pd.to_numeric(x['mkt_cap_weight'])))
valueWeightRet_SIZE_INV=pd.DataFrame(valueWeightRet_SIZE_INV)
valueWeightRet_SIZE_INV=valueWeightRet_SIZE_INV.reset_index(level=['size_portfolio', 'investment_portfolio'])

x=['size_portfolio', 'investment_portfolio']

for x in x:
  valueWeightRet_SIZE_INV[x]=valueWeightRet_SIZE_INV[x].apply(str)


In [ ]:
portfolio_SIZE_INV=valueWeightRet_SIZE_INV
portfolio_SIZE_INV['size_investment_portfolio']=portfolio_SIZE_INV['size_portfolio'] + "/" + portfolio_SIZE_INV['investment_portfolio'] 
portfolio_SIZE_INV=portfolio_SIZE_INV.drop(['size_portfolio', 'investment_portfolio'], axis=1)
portfolio_SIZE_INV=pd.pivot(portfolio_SIZE_INV, columns='size_investment_portfolio', values=0)

portfolio_SIZE_INV

In [ ]:
portfolio_SIZE_INV.columns=['BIG_AGGRESSIVE', 'BIG_CONSERVATIVE', 'BIG_NEUTRAL', 'SMALL_AGGRESSIVE', 'SMALL_CONSERVATIVE', 'SMALL_NEUTRAL']

In [ ]:
#CALCULATE CMA FACTOR

portfolio_SIZE_INV['CMA']= 0.5 * (portfolio_SIZE_INV['SMALL_CONSERVATIVE'] + portfolio_SIZE_INV['BIG_CONSERVATIVE']) - 0.5 * (portfolio_SIZE_INV['SMALL_AGGRESSIVE'] + portfolio_SIZE_INV['BIG_AGGRESSIVE'])

In [ ]:
portfolio_SIZE_INV

# SIZE/VALUE PORTFOLIOS

In [ ]:
##CONSTRUCT SIZE/VALUE PORTFOLIO

valueWeightRet_SIZE_VALUE=main_stocks_df.groupby(['date', 'value_portfolio', 'size_portfolio']).apply(lambda x: np.average(pd.to_numeric(x['ret']), weights=pd.to_numeric(x['mkt_cap_weight'])))
valueWeightRet_SIZE_VALUE=pd.DataFrame(valueWeightRet_SIZE_VALUE)
valueWeightRet_SIZE_VALUE=valueWeightRet_SIZE_VALUE.reset_index(level=['size_portfolio', 'value_portfolio'])

x=['size_portfolio', 'value_portfolio']

for x in x:
  valueWeightRet_SIZE_VALUE[x]=valueWeightRet_SIZE_VALUE[x].apply(str)

In [ ]:
portfolio_SIZE_VALUE=valueWeightRet_SIZE_VALUE
portfolio_SIZE_VALUE['SIZE_VALUEue_portfolio']=portfolio_SIZE_VALUE['size_portfolio'] + "/" + portfolio_SIZE_VALUE['value_portfolio'] 
portfolio_SIZE_VALUE=portfolio_SIZE_VALUE.drop(['size_portfolio', 'value_portfolio'], axis=1)
portfolio_SIZE_VALUE=pd.pivot(portfolio_SIZE_VALUE, columns='SIZE_VALUEue_portfolio', values=0)

portfolio_SIZE_VALUE

In [ ]:
portfolio_SIZE_VALUE.columns=['BIG_VALUE', 'BIG_GROWTH', 'BIG_NEUTRAL_VALUE,', 'SMALL_VALUE', 'SMALL_GROWTH', 'SMALL_NEUTRAL_VALUE']

In [ ]:
#CALCULATE HML FACTOR 

portfolio_SIZE_VALUE['HML']=0.5 * (portfolio_SIZE_VALUE['SMALL_VALUE'] + portfolio_SIZE_VALUE['BIG_VALUE']) - 0.5*(portfolio_SIZE_VALUE['SMALL_GROWTH']+portfolio_SIZE_VALUE['BIG_GROWTH'])


In [ ]:
portfolio_SIZE_VALUE

# CACLULATE THE THREE SMB PORTFOLIOS USING THE PORTFOLIOS ALREADY CONSTRUCTED

In [ ]:
'''
 	


'''

In [ ]:
#USE SIZE/VALUE PORTFOLIO TO CALCULATE SMB_BM
portfolio_SIZE_VALUE['SMB_BM']= 1/3 * (portfolio_SIZE_VALUE['SMALL_VALUE'] + portfolio_SIZE_VALUE['SMALL_NEUTRAL_VALUE'] + portfolio_SIZE_VALUE['SMALL_GROWTH']) - 1/3 * (portfolio_SIZE_VALUE['BIG_VALUE'] + portfolio_SIZE_VALUE['BIG_NEUTRAL_VALUE,'] + portfolio_SIZE_VALUE['BIG_GROWTH'])

#USE SIZE/OP PORTFOLIO TO CALCULATE SMB_OP

portfolio_SIZE_OP['SMB_OP']= 1/3 * (portfolio_SIZE_OP['SMALL_ROBUST'] + portfolio_SIZE_OP['SMALL_NEUTRAL'] + portfolio_SIZE_OP['SMALL_WEAK'] ) - 1/3 * (portfolio_SIZE_OP['BIG_ROBUST'] + portfolio_SIZE_OP['BIG_NEUTRAL'] + portfolio_SIZE_OP['BIG_WEAK'])

#USE SIZE/INVESTMENT PORTFOLIO TO CALCULATE SMB_INV

portfolio_SIZE_INV['SMB_INV']= 1/3 * (portfolio_SIZE_INV['SMALL_CONSERVATIVE'] + portfolio_SIZE_INV['SMALL_NEUTRAL'] + portfolio_SIZE_INV['SMALL_AGGRESSIVE']) - 1/3 *(portfolio_SIZE_INV['BIG_CONSERVATIVE'] + portfolio_SIZE_INV['BIG_NEUTRAL'] + portfolio_SIZE_INV['BIG_AGGRESSIVE']) 

In [ ]:
portfolio_SIZE_VALUE

In [ ]:
portfolio_SIZE_INV

In [ ]:
portfolio_SIZE_OP

In [ ]:
#JOIN DATAFRAMES TOGETHER TO CALCULATE OVERALL SMB

x=[portfolio_SIZE_OP, portfolio_SIZE_VALUE, portfolio_SIZE_INV]

for x in x:
  x['DATE']=x.index

portfolio_SIZE_INV['DATE']=portfolio_SIZE_INV['DATE'].index

In [ ]:
portfolio_SIZE_INV['DATE']=portfolio_SIZE_INV.index

In [ ]:
portfolio_SIZE_OP['DATE']=pd.to_datetime(portfolio_SIZE_OP['DATE'])
portfolio_SIZE_VALUE['DATE']=pd.to_datetime(portfolio_SIZE_VALUE['DATE'])
portfolio_SIZE_INV['DATE']=pd.to_datetime(portfolio_SIZE_INV['DATE'])




In [ ]:
portfolio_SIZE_INV.set_index('DATE', inplace=True)
portfolio_SIZE_OP.set_index('DATE', inplace=True)
portfolio_SIZE_VALUE.set_index('DATE', inplace=True)

In [ ]:
portfolio_SIZE_OP

In [ ]:
portfolio_SIZE_OP.to_sql('op', conn, if_exists='replace')
portfolio_SIZE_VALUE.to_sql('value', conn, if_exists='replace')
portfolio_SIZE_INV.to_sql('inv', conn, if_exists='replace')


query='''
          SELECT DISTINCT op.DATE, op.RMW, op.SMB_OP, value.HML, value.SMB_BM, inv.CMA, inv.SMB_INV 
          FROM op
          LEFT JOIN value on op.DATE=value.DATE
          LEFT JOIN inv on op.DATE=inv.DATE
                 '''

clean_5_factors=pd.read_sql(query, conn)

In [ ]:
clean_5_factors

In [ ]:
clean_5_factors['SMB']=1/3 * (clean_5_factors['SMB_OP'] + clean_5_factors['SMB_BM'] + clean_5_factors['SMB_INV'] )

In [ ]:
clean_5_factors

# CHECKER

In [ ]:
import pandas_datareader.data as web
from pandas_datareader.famafrench import get_available_datasets
datasets = get_available_datasets()

FF_Momentum_Factor=[dataset for dataset in datasets if 'F-F_Research_Data_5_Factors_2x3_daily' in dataset ]

In [ ]:
datasets

In [ ]:
FF_5=web.DataReader(FF_Momentum_Factor[0],'famafrench',start='2019-06-01',end='2022-11-01')[0]

In [ ]:
FF_5['DATE']=FF_5.index
FF_5['DATE']=FF_5['DATE'].apply(str)
FF_5['DATE']=pd.to_datetime(FF_5['DATE'], format="%Y-%m")



In [ ]:
FF_5

In [ ]:
FF_5.set_index('DATE', inplace=True)

In [ ]:
FF_5.to_sql('O', conn, if_exists='replace')

clean_5_factors.to_sql('R', conn, if_exists='replace')

query=''' 
        SELECT DISTINCT R.DATE, R.SMB, R.HML, R.CMA, R.RMW, O.SMB as SMB_FF, O.HML as HML_FF, O.RMW as RMW_FF, O.CMA as CMA_FF
        FROM R
        LEFT JOIN O
        ON R.DATE=O.DATE'''

checker=pd.read_sql(query, conn)

In [ ]:
checker

In [ ]:
checker.corr(method='pearson')